<a href="https://colab.research.google.com/github/min-woo-seong/comin-storia/blob/main/%EA%B5%B0%EC%A7%91%ED%99%94(%EA%B3%A0%EA%B0%9D_%EC%84%B8%EA%B7%B8%EB%A8%BC%ED%85%8C%EC%9D%B4%EC%85%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <고객 세그먼테이션>
##  세그먼테이션의 정의와 기법
### - 고객을 분류하는 요소는 어떤 상품을 얼마나 많은 비용을 써서 얼마나 자주 사용하는가? 에 기반한 정보로 분류하는 것이다.
고객 세그먼테이션의 주요 목표는 "타깃 마케팅" 이다.
타깃 마케팅이란 고객을 여러 특성에 맞게 세분화 해서 유형에 따라 맞춤형 마케팅이나 서비스를 제공하는것 이다.
# RFM
### Recency : 가장 최근 상품 구입일 에서 오늘까지의 기간
### Frequency : 상품 구매 횟수
### Monetary value : 총 구매 금액


In [2]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/00352
retail_df = pd.read_excel(io='/content/drive/MyDrive/Colab Notebooks/Online Retail.xlsx')
retail_df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


* InvoiceNo : 주문번호 'C'로 시작하는것은 취소 주문
* StockCode : 제품코드(Item code)
* Description : 제품 설명
* Quantity :주문 제품 건수
* InvoiceDate : 주문 일자
* UnitPrice : 제품 단가
* CustomerId : 고객 번호
* Country : 국가명

In [17]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


# 1. 결측값 제거

In [19]:
# Null데이터및 오류 데이터 삭제
retail_df = retail_df[retail_df['Quantity'] > 0]
retail_df = retail_df[retail_df['UnitPrice'] > 0]
retail_df = retail_df[retail_df['CustomerID'].notnull()]
print(retail_df.shape)
retail_df.isnull().sum()

(397884, 8)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [21]:
# 영국이 대다수이므로 영국제외 타국가 데이터는 결측값으로 제거
retail_df['Country'].value_counts()[:5]

United Kingdom    354321
Germany             9040
France              8341
EIRE                7236
Spain               2484
Name: Country, dtype: int64

In [22]:
retail_df = retail_df[retail_df['Country']=='United Kingdom']
print(retail_df.shape)

(354321, 8)


# 2. RFM 기반 데이터 가공